In [17]:
import pandas as pd
import numpy as np
from we_eat import build_database, build_user_matrix, clean_database, extract_reviews, ALS_recommender, recommender


In [18]:
bizinfo_df = pd.read_pickle('data/bizinfo_df.pkl')

In [9]:
#compile MongoDB
rest_db = build_database.build_database()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [10]:
#retrieve MongoDB
db = build_database.retrieve_database()

In [11]:
db.head()

,_id,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,5bf5e06f91d56f59f158cfa4,amazon-go-seattle-5,"[{'alias': 'convenience', 'title': 'Convenienc...","{'latitude': 47.60638, 'longitude': -122.33122}",,493.410866,XDfmz7Woxx6NkZDGBtAVAQ,https://s3-media1.fl.yelpcdn.com/bphoto/fDQhxw...,False,"{'address1': '920 5th Ave', 'address2': None, ...",Amazon Go,,NaN,5.0,7,[],https://www.yelp.com/biz/amazon-go-seattle-5?a...
1,5bf5e06f91d56f59f158cfa5,the-pink-door-seattle-4,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 47.61028, 'longitude': -122.3425}",(206) 443-3241,1327.897033,VOPdG8llLPaga9iJxXcMuQ,https://s3-media1.fl.yelpcdn.com/bphoto/c_vzRF...,False,"{'address1': '1919 Post Alley', 'address2': ''...",The Pink Door,+12064433241,$$,4.5,4019,[],https://www.yelp.com/biz/the-pink-door-seattle...
2,5bf5e06f91d56f59f158cfa6,nirmals-seattle,"[{'alias': 'indpak', 'title': 'Indian'}]","{'latitude': 47.60147, 'longitude': -122.33262}",(206) 683-9701,228.364627,GXz21OgpWOtnCF0GDXHPhA,https://s3-media3.fl.yelpcdn.com/bphoto/LVSEXb...,False,"{'address1': '106 Occidental Ave S', 'address2...",Nirmal's,+12066839701,$$,4.0,424,"[restaurant_reservation, pickup]",https://www.yelp.com/biz/nirmals-seattle?adjus...
3,5bf5e06f91d56f59f158cfa7,bad-bishop-seattle,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...","{'latitude': 47.60286, 'longitude': -122.33427}",(206) 623-3440,344.167932,KFMsgY5mV_wCYsa0XX_hEQ,https://s3-media1.fl.yelpcdn.com/bphoto/1GpjX0...,False,"{'address1': '704 1st Ave', 'address2': None, ...",Bad Bishop,+12066233440,NaN,4.5,6,[],https://www.yelp.com/biz/bad-bishop-seattle?ad...
4,5bf5e06f91d56f59f158cfa8,pike-place-chowder-seattle,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 47.60939, 'longitude': -122.34112}",(206) 267-2537,1176.460907,6I28wDuMBR5WLMqfKxaoeg,https://s3-media3.fl.yelpcdn.com/bphoto/ijju-w...,False,"{'address1': '1530 Post Aly', 'address2': 'Ste...",Pike Place Chowder,+12062672537,$$,4.5,6317,[pickup],https://www.yelp.com/biz/pike-place-chowder-se...


In [12]:
#clean database
cleaned_db = clean_database.clean_it_all(db)

/Users/gnishimura/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
cleaned_db.shape

(669, 170)

In [61]:
V = cleaned_db.drop(columns=['id','image_url', 'location', 'rating', 'review_count',
       'transactions', 'url', 'dist_from_galvanize', 'cats', 'popularity'])

In [62]:
V

,category_convenience,category_salad,category_sandwiches,category_italian,category_wine_bars,category_seafood,category_indpak,category_cocktailbars,category_comfortfood,category_soup,...,category_beergardens,category_gelato,category_comedyclubs,category_theater,category_flowers,category_gourmet,$,$$,$$$,$$$$
alias,,,,,,,,,,,,,,,,,,,,,
amazon-go-seattle-5,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
the-pink-door-seattle-4,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
nirmals-seattle,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
bad-bishop-seattle,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
pike-place-chowder-seattle,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
biscuit-bitch-seattle-5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
damn-the-weather-seattle,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
tsukushinbo-seattle,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
taylor-shellfish-oyster-bar-seattle-2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
rr = recommender.RestaurantRecommender()

TypeError: __init__() missing 3 required positional arguments: 'main_database', 'user_matrix', and 'restaurant_matrix'

In [21]:
bizinfo_df.columns

Index(['Accepts Apple Pay', 'Accepts Bitcoin', 'Accepts Credit Cards',
       'Accepts Google Pay', 'Alcohol', 'Ambience', 'Attire', 'Best Nights',
       'Bike Parking', 'Cannabis Products', 'Caters', 'Coat Check', 'Delivery',
       'Dogs Allowed', 'Drive-Thru', 'Gender Neutral Restrooms', 'Good For',
       'Good For Dancing', 'Good for Groups', 'Good for Kids',
       'Good for Working', 'Happy Hour', 'Has Dairy-free Options',
       'Has Gluten-free Options', 'Has Halal Options', 'Has Kosher Options',
       'Has Pool Table', 'Has Soy-free Options', 'Has TV', 'Liked by Vegans',
       'Liked by Vegetarians', 'Music', 'Noise Level',
       'Offers Military Discount', 'Open to All', 'Outdoor Seating', 'Parking',
       'Smoking', 'Take-out', 'Takes Reservations', 'Waiter Service',
       'Wheelchair Accessible', 'Wi-Fi'],
      dtype='object')

In [44]:
bizinfo_df.T

alias,amazon-go-seattle-5,the-pink-door-seattle-4,nirmals-seattle,biscuit-bitch-seattle-5,damn-the-weather-seattle,bad-bishop-seattle,pike-place-chowder-seattle,tsukushinbo-seattle,taylor-shellfish-oyster-bar-seattle-2,elliotts-oyster-house-seattle-2,...,restaurant-specialty-services-seattle,union-deli-la-puget-sound-plaza-building-seattle,measure-seattle,subway-seattle-3,u-bank-cafe-seattle,subway-seattle-31,hamburg-frites-seattle,subway-seattle-66,organic-to-go-seattle-4,chez-dave-seattle-2
Accepts Apple Pay,NaN,No,No,Yes,No,NaN,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accepts Bitcoin,NaN,NaN,No,NaN,NaN,NaN,No,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accepts Credit Cards,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,Yes,Yes,...,NaN,Yes,Yes,Yes,NaN,Yes,NaN,Yes,Yes,Yes
Accepts Google Pay,NaN,No,No,NaN,No,NaN,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcohol,NaN,Full Bar,Full Bar,No,Full Bar,NaN,No,Beer & Wine Only,Full Bar,Full Bar,...,NaN,NaN,Full Bar,No,NaN,No,NaN,No,No,No
Ambience,NaN,Romantic,"Casual, Trendy, Classy","Hipster, Casual, Trendy","Hipster, Casual, Trendy",NaN,"Casual, Touristy",Casual,Casual,Touristy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casual
Attire,NaN,Casual,Casual,Casual,Casual,NaN,Casual,Casual,Casual,Casual,...,NaN,Casual,Dressy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Best Nights,NaN,"Wed, Fri, Sat",NaN,NaN,"Fri, Sat, Sun",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bike Parking,NaN,Yes,Yes,Yes,Yes,NaN,Yes,Yes,No,Yes,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
Cannabis Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
bizinfo_df.replace(to_replace={'Yes': 1, 'No': -1, None: 0})

,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Accepts Google Pay,Alcohol,Ambience,Attire,Best Nights,Bike Parking,Cannabis Products,...,Offers Military Discount,Open to All,Outdoor Seating,Parking,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi
alias,,,,,,,,,,,,,,,,,,,,,
amazon-go-seattle-5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
the-pink-door-seattle-4,-1,0,1,-1,Full Bar,Romantic,Casual,"Wed, Fri, Sat",1,0,...,0,0,1,"Garage, Street",0,-1,1,1,-1,Free
nirmals-seattle,-1,-1,1,-1,Full Bar,"Casual, Trendy, Classy",Casual,0,1,0,...,0,0,1,Street,0,1,1,1,1,Free
biscuit-bitch-seattle-5,1,0,1,0,-1,"Hipster, Casual, Trendy",Casual,0,1,0,...,0,0,-1,Street,0,1,-1,-1,1,Free
damn-the-weather-seattle,-1,0,1,-1,Full Bar,"Hipster, Casual, Trendy",Casual,"Fri, Sat, Sun",1,0,...,0,0,1,Street,-1,-1,-1,1,1,-1
bad-bishop-seattle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pike-place-chowder-seattle,-1,-1,1,-1,-1,"Casual, Touristy",Casual,0,1,0,...,0,0,1,Street,0,1,-1,0,0,Free
tsukushinbo-seattle,-1,-1,1,-1,Beer & Wine Only,Casual,Casual,0,1,0,...,0,0,-1,Street,0,-1,1,1,0,-1
taylor-shellfish-oyster-bar-seattle-2,-1,0,1,-1,Full Bar,Casual,Casual,0,-1,0,...,0,0,-1,Street,0,-1,-1,1,0,-1


## Merge bizinfo_df and cleaned_df to get full set of restaurant features

In [30]:
full_df = pd.merge(left=cleaned_db, right=bizinfo_df, how='outer', left_index=True, right_index=True)

In [31]:
full_df.shape

(669, 213)

In [32]:
full_df

,id,image_url,location,rating,review_count,transactions,url,cats,category_convenience,category_salad,...,Offers Military Discount,Open to All,Outdoor Seating,Parking,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi
alias,,,,,,,,,,,,,,,,,,,,,
13-coins-restaurant-seattle-3,SwIjpF_x-bN4dS0lUY-oZw,https://s3-media3.fl.yelpcdn.com/bphoto/qUaEOb...,"{'address1': '255 S King St', 'address2': '', ...",3.0,198,"[pickup, delivery]",https://www.yelp.com/biz/13-coins-restaurant-s...,"tradamerican,bars,breakfast_brunch",0,0,...,NaN,NaN,Yes,"Garage, Street, Validated",Outdoor Area / Patio Only,Yes,No,Yes,Yes,Free
1st-floor-cafe-seattle,vIXehSnhkF0WiB8hA7nmCQ,,"{'address1': '516 3rd Ave', 'address2': 'Rm C7...",2.0,2,[],https://www.yelp.com/biz/1st-floor-cafe-seattl...,delis,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free
206-burger-company-seattle,J7tIW9kx-ZraM7BHwFafHA,https://s3-media2.fl.yelpcdn.com/bphoto/DWxt0U...,"{'address1': '823 3rd Ave', 'address2': 'Ste 1...",4.0,68,[pickup],https://www.yelp.com/biz/206-burger-company-se...,burgers,0,0,...,NaN,NaN,Yes,Street,NaN,Yes,No,No,Yes,Free
206-burger-company-seattle-2,MpyrQiR9a1QYwNVOnKTOYg,https://s3-media1.fl.yelpcdn.com/bphoto/74RPDq...,"{'address1': '1000 Madison St', 'address2': No...",3.5,69,"[delivery, pickup]",https://www.yelp.com/biz/206-burger-company-se...,burgers,0,0,...,NaN,NaN,No,NaN,NaN,Yes,Yes,NaN,NaN,No
2nd-ave-gifts-and-sundries-seattle,gMEqk9ycs93MAsn4ESrEAg,https://s3-media4.fl.yelpcdn.com/bphoto/-YEl8P...,"{'address1': '1000 2nd Ave', 'address2': '', '...",3.5,3,[],https://www.yelp.com/biz/2nd-ave-gifts-and-sun...,"delis,flowers,convenience",1,0,...,NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN,NaN
4th-ave-espresso-bar-seattle-3,BSBzN5y10_kEGDERK26tDA,https://s3-media2.fl.yelpcdn.com/bphoto/Z84_iC...,"{'address1': '411 University St', 'address2': ...",4.5,15,[],https://www.yelp.com/biz/4th-ave-espresso-bar-...,"cafes,macarons,bakeries",0,0,...,NaN,NaN,No,"Valet, Street",NaN,No,No,No,Yes,Free
4th-avenue-caffe-seattle-2,sG-R-Z98lkyRr9qOrIR5JQ,https://s3-media3.fl.yelpcdn.com/bphoto/pIw9so...,"{'address1': '500 4th Ave', 'address2': '', 'a...",2.5,9,[],https://www.yelp.com/biz/4th-avenue-caffe-seat...,"coffee,bakeries,sandwiches",0,0,...,NaN,NaN,Yes,NaN,NaN,Yes,NaN,NaN,NaN,Free
663-bistro-seattle,rKZ3yW2t-y8x2bM4TAGksg,https://s3-media2.fl.yelpcdn.com/bphoto/-lwsSR...,"{'address1': '663 S Weller St', 'address2': ''...",3.0,338,"[pickup, delivery]",https://www.yelp.com/biz/663-bistro-seattle?ad...,"bbq,cantonese,seafood",0,0,...,NaN,NaN,No,Street,NaN,Yes,No,Yes,NaN,No
90-karaoke-seattle-2,X3Al8TPGIQO06ZCLTJuM6A,https://s3-media2.fl.yelpcdn.com/bphoto/V-if16...,"{'address1': '207 2nd Ave S', 'address2': None...",3.0,7,[],https://www.yelp.com/biz/90-karaoke-seattle-2?...,"karaoke,cajun,seafood",0,0,...,Yes,NaN,No,Street,Outdoor Area / Patio Only,Yes,Yes,NaN,Yes,NaN


## Import data from review scrape

In [66]:
als_df = pd.read_pickle('data/als_df.pkl')

In [67]:
als_df.reset_index(drop=True, inplace=True)

In [68]:
als_df = als_df[['user_id', 'item_id', 'rating', 'date']]

In [69]:
als_df.shape

(5216, 4)

## Spark ALS

In [63]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

In [64]:
spark = SparkSession.builder.getOrCreate()

In [70]:
spark_als_df = spark.createDataFrame(als_df) 

In [71]:
pandas_als_df = spark_als_df.toPandas()

In [73]:
pandas_als_df.head()

,user_id,item_id,rating,date
0,1520,596,3.0,2005-08-02
1,1520,592,4.0,2005-09-14
2,1369,480,4.0,2006-05-13
3,1369,601,5.0,2006-05-19
4,1369,488,5.0,2006-05-22


In [74]:
pandas_als_df = pandas_als_df[['user_id', 'item_id', 'rating']]

In [78]:
spark_als_df = spark.createDataFrame(pandas_als_df)

In [79]:
als_model = ALS(
    itemCol='item_id',
    userCol='user_id',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.1,
    rank=10,
    coldStartStrategy = 'drop') 

In [80]:
recommender = als_model.fit(spark_als_df)

## Find the Latent Features Matrices

In [83]:
uf_df = recommender.userFactors.toPandas()

In [84]:
if_df = recommender.itemFactors.toPandas()

In [86]:
user_factors_array = np.array(uf_df['features'].tolist())

In [87]:
item_factors_array = np.array(if_df['features'].tolist())

## Recommendations based on Minimizing Dissatisfaction

### Start with small example

In [89]:
user1 = 'gabe'
user2 = 'nicole'

In [148]:
preds_db = pd.DataFrame(data=[[1, 5, 7, 3, 8, 7, 5, 4], [4, 8, 9, 4, 8, 6, 2, 9]], columns=['rest1', 'rest2', 'rest3', 'rest4', 'rest5', 'rest6', 'rest7', 'rest8'], index=[user1, user2])

In [149]:
preds_db = preds_db.T

In [150]:
preds_db['mean'] = preds_db.mean(axis=1)

In [151]:
preds_db

,gabe,nicole,mean
rest1,1,4,2.5
rest2,5,8,6.5
rest3,7,9,8.0
rest4,3,4,3.5
rest5,8,8,8.0
rest6,7,6,6.5
rest7,5,2,3.5
rest8,4,9,6.5


In [155]:
preds_db_sorted = preds_db.sort_values(by='mean')
preds_db_sorted

,gabe,nicole,mean
rest1,1,4,2.5
rest4,3,4,3.5
rest7,5,2,3.5
rest2,5,8,6.5
rest6,7,6,6.5
rest8,4,9,6.5
rest3,7,9,8.0
rest5,8,8,8.0


In [165]:
preds_db_sorted.loc['rest1'].min()

1.0

In [168]:
def highest_min(rest1, rest2, db):
    if db.loc[rest1].min() > db.loc[rest2].min():
        return rest1
    return rest2

In [169]:
highest_min('rest6', 'rest8', preds_db_sorted)

'rest6'

In [185]:
len(preds_db_sorted.index)

8

In [186]:
def add_resorted_column(preds_db_sorted):
    resorted = []
    for i, rest in enumerate(preds_db_sorted.index):
        if i < len(preds_db_sorted.index):
            print(preds_db_sorted.iloc[i]['mean'])
            if preds_db_sorted.iloc[i]['mean'] != preds_db_sorted.iloc[i+1]['mean']:
                resorted.append(rest)
            else:
                resorted.append(highest_min(preds_db_sorted.index[i], 
                                            preds_db_sorted.index[i+1], 
                                            preds_db_sorted))
        else:
            pass
    return resorted

            


In [187]:
add_resorted_column(preds_db_sorted=preds_db_sorted)

2.5
3.5
3.5
6.5
6.5
6.5
8.0
8.0


IndexError: single positional indexer is out-of-bounds

In [139]:
preds_db['sorted'] = (lambda x: preds_db.index[x] if preds_db['mean'].loc(x + 1) == )

SyntaxError: invalid syntax (<ipython-input-139-67b21baf5d37>, line 1)

In [140]:
preds_db

,gabe,nicole,mean
rest1,1,4,2.5
rest2,5,8,6.5
rest3,7,9,8.0
rest4,3,4,3.5
rest5,8,8,8.0
rest6,7,6,6.5
rest7,5,2,3.5


In [138]:
for rest in preds_db.index:
    if rest['mean'] len(matches[rest]==1):
        preds_db['sorted'] = rest
        
    

KeyError: 'rest1'

In [136]:
matches = {}
for rest, rating in ratings.items():
    if rating in matches:
        matches[rating].append(rest)
    else:
        matches[rating] = [rest] 

matches

{2.5: ['rest1'],
 3.5: ['rest4', 'rest7'],
 6.5: ['rest2', 'rest6'],
 8.0: ['rest3', 'rest5']}

In [122]:
preds_db[preds_db['mean'].duplicated(keep=False)]

,gabe,nicole,mean
rest2,5,8,6.5
rest3,7,9,8.0
rest4,3,4,3.5
rest5,8,8,8.0
rest6,7,6,6.5
rest7,5,2,3.5


In [ ]:
def min_dissat(user1, user2, preds_db):
    for rest in preds_db:
        if 

## Discover Meaning of Latent Features